# 🚀 Instalación y configuración de PySpark en Google Colab
En esta primera parte vamos a instalar **PySpark** en Google Colab.  
Esto nos permitirá simular un entorno distribuido de procesamiento de datos, sin necesidad de montar un cluster real.  

Spark funciona bajo una arquitectura **Driver–Workers**:  
- **Driver:** coordina el trabajo.  
- **Workers:** procesan partes de los datos en paralelo.  

👉 Con estas líneas de código, tendremos un mini-cluster Spark en Colab.


# 🚀 SparkSession y funciones `F` en PySpark

## 🔹 SparkSession
En PySpark, el **punto de entrada principal** para trabajar con datos es la **SparkSession**.  

- **En entornos locales (tu computador o un servidor sin Spark preconfigurado):**  
  Necesitamos **crear la sesión** manualmente, indicando un nombre para nuestra aplicación.  

```python
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ClaseBigData") \
    .getOrCreate()


In [0]:
# Instalamos PySpark
!pip install pyspark

# Importamos librerías
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Creamos la sesión de Spark
spark = SparkSession.builder.appName("ClaseBigData").getOrCreate()

# Verificamos la versión
print("Versión de Spark:", spark.version)

# 📦 `pyspark.sql.functions as F`

## 🔹 ¿Qué es `F`?
En PySpark, la librería `pyspark.sql.functions` (comúnmente importada como `F`) es un **conjunto de funciones listas para usar** que permiten manipular columnas de un **DataFrame distribuido**.

```python
from pyspark.sql import functions as F

# 📌 Funciones más usadas en PySpark (`pyspark.sql.functions as F`)

A continuación se presentan las principales categorías de funciones de `F` con ejemplos de uso en PySpark:

# Guía de Funciones PySpark

## 1. 📌 Operaciones sobre columnas

Permiten crear, seleccionar o condicionar columnas.

```python
# Seleccionar una columna
F.col("columna")

# Crear una columna con un valor fijo
F.lit(100)

# Condicional tipo if-else
F.when(F.col("precio") > 100, F.col("precio") * 0.9).otherwise(F.col("precio"))
```

**Ejemplo de uso:**

```python
df = df.withColumn("con_descuento",
                   F.when(F.col("precio") > 100, F.col("precio") * 0.9)
                    .otherwise(F.col("precio")))
```

## 2. 📊 Agregaciones

Se usan junto a `groupBy()` y `agg()` para resumir información.

```python
F.count("*")                # Contar filas
F.sum("columna")            # Suma
F.avg("columna")            # Promedio
F.max("columna")            # Máximo
F.min("columna")            # Mínimo
F.countDistinct("columna")  # Valores únicos
```

**Ejemplo de uso:**

```python
df.groupBy("categoria").agg(F.avg("precio").alias("precio_promedio"))
```

**Comparación en Python:**

```python
df.groupby("categoria")["precio"].mean()
```

## 3. 🔤 Funciones de texto

Se utilizan para limpiar o manipular cadenas.

```python
F.upper("columna")                                # Convertir a mayúsculas
F.lower("columna")                                # Convertir a minúsculas
F.length("columna")                               # Longitud de la cadena
F.concat(F.col("nombre"), F.lit(" "), F.col("apellido"))  # Concatenar
```

**Ejemplo de uso:**

```python
df = df.withColumn("nombre_completo",
                   F.concat(F.col("nombre"), F.lit(" "), F.col("apellido")))
```

**Comparación en Python:**

```python
df["nombre_completo"] = df["nombre"] + " " + df["apellido"]
```

## 4. 📅 Funciones de fechas y tiempos

Son clave para análisis temporal.

```python
F.current_date()                           # Fecha actual
F.current_timestamp()                      # Fecha y hora actual
F.year("columna_fecha")                    # Extraer año
F.month("columna_fecha")                   # Extraer mes
F.dayofmonth("columna_fecha")              # Extraer día del mes
F.datediff(F.current_date(), "columna_fecha")  # Diferencia en días
```

**Ejemplo de uso:**

```python
df = df.withColumn("edad_dias",
                   F.datediff(F.current_date(), F.col("fecha_nacimiento")))
```

**Comparación en Python:**

```python
(df["fecha_actual"] - df["fecha_nacimiento"]).dt.days
```

## 5. ⚠️ Manejo de nulos

Muy importantes para limpiar datos faltantes.

```python
F.isnull("columna")                 # Detectar nulos True/False
F.coalesce("columna1", "columna2", "columna3", ...)   # Por cada fila, evalúa las columnas en orden y devuelve el primer valor que no sea nulo
```

**Ejemplo de uso:**

```python
df = df.withColumn("valor_final", F.coalesce("columnaA", "columnaB"))
```

**Comparación en Python:**

```python
df["valor_final"] = df["columnaA"].fillna(df["columnaB"])
```


📂 Cargando un CSV masivo en Spark  
En esta sección vamos a cargar un dataset de **15,747,461 registros**, que en Excel o pandas sería prácticamente imposible de manejar.  

Spark puede leer este archivo en paralelo, lo que significa que lo divide en **particiones** y asigna cada fragmento a un *worker*.  
De esta manera, el procesamiento es mucho más eficiente que con herramientas tradicionales.  

👉 Usaremos un CSV de 15,7 millones de filas con el siguiente esquema:

In [0]:
# 1. Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Definir la ruta al archivo en Drive
# Supongamos que el archivo está en "Mi unidad/2_FACT_1_EMPRESA.csv"
file_path = "/content/drive/MyDrive/Curso - Big Data EAN/2_FACT_1_EMPRESA.csv"

# Leemos el CSV en Spark
df = spark.read.csv(file_path, header=True, inferSchema=True, sep=";")

######################################################################
# Lista de columnas actuales
cols = df.columns

# Generamos nombres genéricos: col1, col2, ...
new_cols = [f"col{i+1}" for i in range(len(cols))]

# Creamos un DataFrame con columnas renombradas
df_renamed = df.toDF(*new_cols)

# Ahora eliminamos la columna que originalmente era "Proveedor"
# (en el esquema original estaba en la posición 9, o sea col9)
df = df_renamed.drop("col9")
######################################################################

# Revisamos el esquema del DataFrame
df.printSchema()

# Mostramos algunas filas
df.show(5)

# ⚡ Procesamiento en paralelo con Spark
Ahora vamos a ejecutar una acción muy sencilla: contar el número de registros (`df.count()`).  

En pandas:  
- Se cargaría todo en memoria.  
- Con datasets grandes, la máquina se puede quedar sin RAM.  

En Spark:  
- Cada **worker** cuenta una parte del dataset.  
- El **driver** suma los resultados parciales.  
- Esto se llama **procesamiento distribuido**.  

👉 Imaginemos que tenemos una fila de cajeros contando facturas en paralelo.  
Cada cajero cuenta un bloque, y al final el jefe suma los totales.  
Ese “jefe” es el **Driver** de Spark.


In [0]:
# Contamos los registros de forma distribuida
registros = df.count()
print("Número total de registros:", registros)

🎯 Ejemplo adicional: Agrupación por tipo de registro  
Para mostrar más claro el poder de Spark, agruparemos los datos por **col5**  
y contaremos cuántos registros existen en cada categoría.  

Esto en Excel sería muy lento con más de 15 millones de filas.  
Spark lo hace en paralelo → cada *worker* procesa un bloque y luego los une de forma eficiente.  

In [0]:
# Agrupar por tipo de registro y contarlas
df_group_tipo = df.groupBy("col5") \
                  .agg(F.count("*").alias("total_registros")) \
                  .orderBy(F.col("total_registros").desc())

df_group_tipo.show()